In [1]:
include("lib/all.jl");

In [2]:
particles = make_particles(2^20);
t = Tree(particles, 16);

In [6]:
@time group!(t);

  0.088531 seconds (4 allocations: 160 bytes)


In [7]:
@time inform!(t);

  0.102931 seconds (835.20 k allocations: 12.744 MB, 44.29% gc time)


In [28]:
test_in_cell_mass(t)

In [29]:
ax = zeros(length(particles))
ay = zeros(length(particles))
az = zeros(length(particles));

In [34]:
@time interact!(t, 0.5, ax, ay, az);

  1.810878 seconds (4.81 M allocations: 1.506 GB, 1.86% gc time)
